## BERT Tutorial
### How to leverage BERT models for NLP use cases

<img src='https://towardsml.files.wordpress.com/2019/09/bert.png?w=1400' width=450>

#### A lot of available models - choose according to computational powers

Link: https://github.com/google-research/bert/

<img src='data/bert_models.png' width=600>


#### I will use DistilBert - smaller BERT that reaches similarly good performance level

DistilBert
- 66m parameters (Bert 110m)
- Layers / Hidden dimensions / Attention heads: 6 / 768 / 12 (BERT: 12 / 768 / 12)
- Performance: 97% of BERT

Complete documentation: https://huggingface.co/docs/transformers/model_doc/distilbert#distilbert (actually very user friendly)

In [1]:
import sys
import torch
from transformers import __version__ as transformers_version

print('Python version:', sys.version)
print('PyTorch version:', torch.__version__)
print('Transformers version:', transformers_version)

Python version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 1.10.2
Transformers version: 4.17.0


In [10]:
torch.cuda.is_available()

C:\Users\krist\anaconda3\lib\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
#!pip install transformers
torch.set_num_threads(4)

from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification, pipeline

- DistilBertTokenizer: tokenizes input sequence
- DistilBertModel: creates embeddings on top of tokenized sequence (DistilBertTokenizer + training embeddings)
    - if using TensorFlow: TFDistilBertModel
- DistilBertForSequenceClassification: already builds a classifier on top of embeddings (DistilBertModel + classifier)
    - if using TensorFlow: TFDistilBertForSequenceClassification
- pipeline: DIY use cases    

## 1. Try out [MASK] performance

In [3]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')

Downloading: 100%|██████████| 256M/256M [00:42<00:00, 6.24MB/s] 


In [4]:
unmasker('She wanted to go to [MASK].', top_k = 5)

[{'score': 0.10261113196611404,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'she wanted to go to sleep.'},
 {'score': 0.06890519708395004,
  'token': 6014,
  'token_str': 'heaven',
  'sequence': 'she wanted to go to heaven.'},
 {'score': 0.055472031235694885,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'she wanted to go to bed.'},
 {'score': 0.029796097427606583,
  'token': 7173,
  'token_str': 'jail',
  'sequence': 'she wanted to go to jail.'},
 {'score': 0.02460319548845291,
  'token': 2267,
  'token_str': 'college',
  'sequence': 'she wanted to go to college.'}]

In [5]:
unmasker("I can't find my [MASK] .", top_k = 5)

[{'score': 0.04563814774155617,
  'token': 21714,
  'token_str': 'bearings',
  'sequence': "i can't find my bearings."},
 {'score': 0.029267525300383568,
  'token': 3042,
  'token_str': 'phone',
  'sequence': "i can't find my phone."},
 {'score': 0.0241349246352911,
  'token': 3437,
  'token_str': 'answer',
  'sequence': "i can't find my answer."},
 {'score': 0.023151548579335213,
  'token': 6998,
  'token_str': 'answers',
  'sequence': "i can't find my answers."},
 {'score': 0.02219867706298828,
  'token': 3611,
  'token_str': 'dad',
  'sequence': "i can't find my dad."}]

In [6]:
unmasker("I wish I had a [MASK].", top_k = 5)

[{'score': 0.08036588877439499,
  'token': 6898,
  'token_str': 'boyfriend',
  'sequence': 'i wish i had a boyfriend.'},
 {'score': 0.04185354337096214,
  'token': 3336,
  'token_str': 'baby',
  'sequence': 'i wish i had a baby.'},
 {'score': 0.030962644144892693,
  'token': 6513,
  'token_str': 'girlfriend',
  'sequence': 'i wish i had a girlfriend.'},
 {'score': 0.024552473798394203,
  'token': 3382,
  'token_str': 'chance',
  'sequence': 'i wish i had a chance.'},
 {'score': 0.020696187391877174,
  'token': 3959,
  'token_str': 'dream',
  'sequence': 'i wish i had a dream.'}]

In [7]:
unmasker("The black woman worked as a [MASK].")

[{'score': 0.13283944129943848,
  'token': 13877,
  'token_str': 'waitress',
  'sequence': 'the black woman worked as a waitress.'},
 {'score': 0.12586164474487305,
  'token': 6821,
  'token_str': 'nurse',
  'sequence': 'the black woman worked as a nurse.'},
 {'score': 0.11708816140890121,
  'token': 10850,
  'token_str': 'maid',
  'sequence': 'the black woman worked as a maid.'},
 {'score': 0.11500067263841629,
  'token': 19215,
  'token_str': 'prostitute',
  'sequence': 'the black woman worked as a prostitute.'},
 {'score': 0.0472276546061039,
  'token': 22583,
  'token_str': 'housekeeper',
  'sequence': 'the black woman worked as a housekeeper.'}]

In [8]:
unmasker("The white man worked as a [MASK].")

[{'score': 0.12353681027889252,
  'token': 20987,
  'token_str': 'blacksmith',
  'sequence': 'the white man worked as a blacksmith.'},
 {'score': 0.10142574459314346,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the white man worked as a carpenter.'},
 {'score': 0.049850210547447205,
  'token': 7500,
  'token_str': 'farmer',
  'sequence': 'the white man worked as a farmer.'},
 {'score': 0.03932555019855499,
  'token': 18594,
  'token_str': 'miner',
  'sequence': 'the white man worked as a miner.'},
 {'score': 0.033517707139253616,
  'token': 14998,
  'token_str': 'butcher',
  'sequence': 'the white man worked as a butcher.'}]

In [9]:
unmasker("Black people are [MASK].")

[{'score': 0.056957632303237915,
  'token': 12421,
  'token_str': 'excluded',
  'sequence': 'black people are excluded.'},
 {'score': 0.032912153750658035,
  'token': 22216,
  'token_str': 'enslaved',
  'sequence': 'black people are enslaved.'},
 {'score': 0.0325375571846962,
  'token': 8135,
  'token_str': 'christians',
  'sequence': 'black people are christians.'},
 {'score': 0.02683640830218792,
  'token': 14302,
  'token_str': 'minorities',
  'sequence': 'black people are minorities.'},
 {'score': 0.017561351880431175,
  'token': 27666,
  'token_str': 'persecuted',
  'sequence': 'black people are persecuted.'}]

## 2. Get features (embeddings) of tokens

In [14]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer

PreTrainedTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

return_tensors (`str` or [`~file_utils.TensorType`], *optional*):
        If set, will return tensors instead of list of python integers. Acceptable values are:

        - `'tf'`: Return TensorFlow `tf.constant` objects.
        - `'pt'`: Return PyTorch `torch.Tensor` objects.
        - `'np'`: Return Numpy `np.ndarray` objects.

In [15]:
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
model.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

In [18]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [22]:
model.embeddings

Embeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [35]:
text = ["I went to the river bank and just laid there.", 
        "I work at an investment bank in New York.", 
        "Do you want to go with me to the bank?"]
encoded_input = tokenizer(text, return_tensors='pt', padding = True)

In [36]:
encoded_input

{'input_ids': tensor([[ 101, 1045, 2253, 2000, 1996, 2314, 2924, 1998, 2074, 4201, 2045, 1012,
          102],
        [ 101, 1045, 2147, 2012, 2019, 5211, 2924, 1999, 2047, 2259, 1012,  102,
            0],
        [ 101, 2079, 2017, 2215, 2000, 2175, 2007, 2033, 2000, 1996, 2924, 1029,
          102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [37]:
encoded_input['input_ids']

tensor([[ 101, 1045, 2253, 2000, 1996, 2314, 2924, 1998, 2074, 4201, 2045, 1012,
          102],
        [ 101, 1045, 2147, 2012, 2019, 5211, 2924, 1999, 2047, 2259, 1012,  102,
            0],
        [ 101, 2079, 2017, 2215, 2000, 2175, 2007, 2033, 2000, 1996, 2924, 1029,
          102]])

In [38]:
encoded_input['input_ids'].numpy()

array([[ 101, 1045, 2253, 2000, 1996, 2314, 2924, 1998, 2074, 4201, 2045,
        1012,  102],
       [ 101, 1045, 2147, 2012, 2019, 5211, 2924, 1999, 2047, 2259, 1012,
         102,    0],
       [ 101, 2079, 2017, 2215, 2000, 2175, 2007, 2033, 2000, 1996, 2924,
        1029,  102]], dtype=int64)

In [39]:
encoded_input['attention_mask'].numpy()

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

How does the tokenized text look like?

In [40]:
for i in range(encoded_input['input_ids'].shape[0]):
    print(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][i])) 

['[CLS]', 'i', 'went', 'to', 'the', 'river', 'bank', 'and', 'just', 'laid', 'there', '.', '[SEP]']
['[CLS]', 'i', 'work', 'at', 'an', 'investment', 'bank', 'in', 'new', 'york', '.', '[SEP]', '[PAD]']
['[CLS]', 'do', 'you', 'want', 'to', 'go', 'with', 'me', 'to', 'the', 'bank', '?', '[SEP]']


In [41]:
for i in range(encoded_input['input_ids'].shape[0]):
    print(len(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][i])) )

13
13
13


How is the output stored?

In [42]:
output = model(**encoded_input)

In [43]:
output[0].shape

torch.Size([3, 13, 768])

In [44]:
output[0][0].shape

torch.Size([13, 768])

In [45]:
output[0][1].shape

torch.Size([13, 768])

In [46]:
output[0][2].shape

torch.Size([13, 768])

In [48]:
# can convert to numpy
output[0][0].detach().numpy().shape

(13, 768)

Words with multiple meanings

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [50]:
bank_river = output[0][0][6]
bank_financial = output[0][1][6]
bank_universal = output[0][2][10]

In [51]:
bank_matrix = np.concatenate((bank_river.detach().numpy().reshape(1, 768), 
                              bank_financial.detach().numpy().reshape(1, 768), 
                              bank_universal.detach().numpy().reshape(1, 768)))

In [52]:
pd.DataFrame(cosine_similarity(bank_matrix), 
             columns=['River', 'Investment', 'Universal'],
             index=['River', 'Investment', 'Universal'])

,River,Investment,Universal
River,1.000000,0.688735,0.770934
Investment,0.688735,1.000000,0.836289
Universal,0.770934,0.836289,1.000000


In [53]:
text = ["My date went great last night!", 
        "What's today's date?", 
        "This date is too sour to eat."]
encoded_input = tokenizer(text, return_tensors='pt', padding = True)

for i in range(encoded_input['input_ids'].shape[0]):
    print(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][i])) 
    
output = model(**encoded_input)

date_rel = output[0][0][2]
date_time = output[0][1][7]
date_food = output[0][2][2]

date_matrix = np.concatenate((date_rel.detach().numpy().reshape(1, 768), 
                              date_time.detach().numpy().reshape(1, 768), 
                              date_food.detach().numpy().reshape(1, 768)))

pd.DataFrame(cosine_similarity(date_matrix), 
             columns=['Relationship', 'Calendar', 'Food'],
             index=['Relationship', 'Calendar', 'Food'])

['[CLS]', 'my', 'date', 'went', 'great', 'last', 'night', '!', '[SEP]', '[PAD]']
['[CLS]', 'what', "'", 's', 'today', "'", 's', 'date', '?', '[SEP]']
['[CLS]', 'this', 'date', 'is', 'too', 'sour', 'to', 'eat', '.', '[SEP]']


,Relationship,Calendar,Food
Relationship,1.000000,0.825439,0.743566
Calendar,0.825439,1.000000,0.771033
Food,0.743566,0.771033,1.000000


## 3. Transfer learning without fine tuning - sentiment classification

<img src='https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification-example.png' width=800>


In [65]:
# already loaded
# import torch
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [54]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', 
                 delimiter='\t', header=None)
df.columns = ['review', 'label']

df.head()

,review,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


#### 1. Tokenization

In [55]:
%%time 
tokenized = df['review'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Wall time: 6.2 s


In [56]:
print('Max length:', tokenized.map(len).max())
print('Median length:', tokenized.map(len).median())
print('Mean length:', tokenized.map(len).mean())

Max length: 67
Median length: 22.0
Mean length: 23.341907514450867


Use tokenizer function that creates padded embeddings and outputs attention masks (what to consider, what not to consider)

In [57]:
MAX_LEN = 30

def bert_tokenizer(text):
    
    encoded_text = tokenizer.encode_plus(text,  max_length = MAX_LEN, truncation=True,  padding='max_length',  
                                         return_attention_mask=True, return_tensors='tf')
    
    return encoded_text['input_ids'][0].numpy(), encoded_text['attention_mask'][0].numpy()

In [58]:
bert_tokenizer('Sample test that will be padded')

(array([  101,  7099,  3231,  2008,  2097,  2022, 20633,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]))

In [59]:
%%time

tokenized_padded, attention_masks = zip(*df['review'].apply(lambda x: bert_tokenizer(x)))

Wall time: 13.1 s


In [75]:
input_ids = torch.tensor(np.array(tokenized_padded))  # could add requires_grad=False in torch, torch.no_grad() is more universal
attention_mask = torch.tensor(np.array(attention_masks))

print(input_ids.shape)
print(attention_mask.shape)

torch.Size([6920, 30])
torch.Size([6920, 30])


#### 2. Apply BERT on tokens

In [94]:
%%time

with torch.no_grad(): # no need to keep track of gradients
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Wall time: 8min 8s


In [95]:
last_hidden_states[0].shape

torch.Size([6920, 30, 768])

6920 sentences, 30 words (tokens) in each sentence, 768 dimensions for each word (token)

#### 3. Get sentence embeddings out of the resuling tensor


<img src='https://camo.githubusercontent.com/6c2185c7620a3fe52f1968752febb6467723f4485c257442d3b0ed03bb0da197/68747470733a2f2f6a616c616d6d61722e6769746875622e696f2f696d616765732f64697374696c424552542f626572742d6f75747075742d74656e736f722d73656c656374696f6e2e706e67' width=1000>


In [96]:
X = last_hidden_states[0][:,0,:].numpy()
y = df['label']

print(X.shape)
print(y.shape)

(6920, 768)
(6920,)


#### 4. Fit model, evaluate

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 91, train_size = 0.8)

In [99]:
logit = LogisticRegression(max_iter = 1000).fit(X_train, y_train)

In [100]:
y_pred_class = logit.predict(X_test)
y_pred_prob = logit.predict_proba(X_test)[:, 1]

In [101]:
print('Ratio of positive class:', y.value_counts()[1] / df.shape[0])
print('Accuracy:', accuracy_score(y_test, y_pred_class))
print('AUC:', roc_auc_score(y_test, y_pred_prob))

Ratio of positive class: 0.5216763005780347
Accuracy: 0.8345375722543352
AUC: 0.9162325654286162


#### 5. Predict sentiment of any text

In [102]:
def predict_sentiment(text):
    
    _tokenized, _attention_mask = bert_tokenizer(text)

    _tokenized = torch.reshape(torch.from_numpy(_tokenized), (1, 30))
    _attention_mask = torch.reshape(torch.from_numpy(_attention_mask), (1, 30))
    _last_hidden_state = model(_tokenized, attention_mask = _attention_mask)
    _X = _last_hidden_state[0][:,0,:][0].detach().numpy().reshape(1, -1)

    #predicted_class = logit.predict(_X)[0]
    predicted_proba = logit.predict_proba(_X)[:, 1][0]

    return print('Probability of being positive:', predicted_proba)

In [103]:
text = 'I though the movie was going to suck, but actually it turned out to be really good.'
predict_sentiment(text)

Probability of being positive: 0.4585272365085751


In [104]:
text = 'Overall OK, nothing special'
predict_sentiment(text)

Probability of being positive: 0.21126795797738587


In [105]:
text = 'Liked it'
predict_sentiment(text)

Probability of being positive: 0.8487172901582756


In [106]:
text = 'What a fucking amazing picture'
predict_sentiment(text)

Probability of being positive: 0.6141236449664051


In [107]:
text = 'What a fucking amazing picture!'
predict_sentiment(text)

Probability of being positive: 0.6883185167869081


## 4. Fine tuning a DistilBert model